In [1]:
import myNHLstats as mns
from myNHLstats import GamePlot, GameMap
from nhlstats import *

import os
import pandas as pd
import datetime as dt
from datetime import *

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import clear_output

from IPython.display import display, Javascript

disable_js = """
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
"""
display(Javascript(disable_js))

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

<IPython.core.display.Javascript object>

In [2]:
style = {'description_width': 'initial'}
DatePicker = widgets.DatePicker(
    description='Pick a Date',
    disabled=False,
    value= date.today(),
    style=style
)
def lst_unique(x):
    return list(dict.fromkeys(x))
def SelectGames(GameDay):

    # Download data to pandas DF
    try:
        games = pd.DataFrame(list_games(GameDay, GameDay))
        games = games.drop(columns=['season'])
    except:
        clear_output(wait=True)
        return None, None
    
    if len(games) > 0:

        #Create lists for all teams playing, and all game_id's
        teams = []
        #game_ids = []
        [teams.append(i) for i in games.away_team]
        [teams.append(i) for i in games.home_team]
        #[game_ids.append(i) for i in games.game_id]

        teams.sort()

        teams = lst_unique(teams)

    team_select = widgets.Dropdown(
        options=teams,
        value=teams[0],
        description='Team Name: ',
        disabled=False,
        style=style
    )
    return games, team_select

goButton = widgets.Button(value=False,
                          description='Create Plots',
                          disabled=False,
                          button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                          tooltip='Description',
                          icon='check', # (FontAwesome names without the `fa-` prefix)
                          style=style)
def click(b):
    
    gameid = games.loc[(games.home_team == team_select.value) | (games.away_team == team_select.value), 'game_id'].max()
    #clear_output(wait=False)
    
    # Create game plot
    gp = GamePlot(gameid)

    GamePlot.ModifyDFs(gp)
    GamePlot.AggregateData(gp)
    GamePlot.Prior5Shots(gp)
    GamePlot.Prior5Hits(gp)
    GamePlot.MenOnIce(gp)
    GamePlot.GoalsDF(gp)
    GamePlot.SumDF(gp)
    display(gp.SumStatsStyle)
    
    GamePlot.Plot(gp, AltColor=[True,True])
    GPlot = gp.GPlot
    
    # Create shots/goals map
    gm = GameMap(gameid)

    GameMap.XY_SnG(gm)
    GameMap.XY_Plays(gm)
    GameMap.Map(gm, AltColor=[True,True], shots=True)
    Map1 = gm.Map
    
    # Create hits, TA's, GA's map
    gm = GameMap(gameid)

    GameMap.XY_SnG(gm)
    GameMap.XY_Plays(gm)
    GameMap.Map(gm, AltColor=[True,True], shots=False)
    Map2 = gm.Map
    
    
goButton.on_click(click)    
games, team_select = SelectGames(DatePicker.value)


In [3]:
@interact
def change(DP=DatePicker, TS=team_select):
    global games
    games, temp = SelectGames(DatePicker.value)
    
    try:
        teams = []
        [teams.append(i) for i in games.away_team]
        [teams.append(i) for i in games.home_team]
    
        teams.sort()
        teams = lst_unique(teams)
        team_select.options = teams
    
    except:
        team_select.options =['NO GAMES']
        
    clear_output(wait=True)
    display(games)
    display(goButton)

interactive(children=(DatePicker(value=datetime.date(2020, 9, 4), description='Pick a Date', style=Description…

gp.plays

gp = GamePlot(2019030243)

GamePlot.ModifyDFs(gp)
GamePlot.AggregateData(gp)
GamePlot.Prior5Shots(gp)
GamePlot.Prior5Hits(gp)
GamePlot.MenOnIce(gp)
GamePlot.GoalsDF(gp)
GamePlot.SumDF(gp)

gp.SumStatsStyle

GamePlot.Plot(gp, AltColor=[True,True])

gm = GameMap(2019030243)

GameMap.XY_SnG(gm)
GameMap.XY_Plays(gm)
GameMap.Map(gm, AltColor=[True,True], shots=True)

AllDFs = []
path = "C:\\Users\\grega\\Google Drive\\Python\\NHLstats\\2019-2020 Season Stats\\COL"
for subdir, dirs, files in os.walk(path):
    for file in files:
        if file[4:14] != 'Master.csv':
            
            gameid = int(file[4:14])
            #print(gameid)
            gm = GameMap(gameid)
            GameMap.XY_SnG(gm)

            if gm.HomeAbrv == 'COL':
                df = gm.HomeG.copy()

            else:
                df = gm.AwayG.copy()
            if len(df) > 0:
                df.loc[:,'MarkerStyle'] = 'o'
                df.loc[:, 'x'] = -1 * abs(df.loc[(df.period == 2), 'x'])
                df.loc[:, 'y'] = -1 * df.loc[(df.period == 2), 'y']
                df.loc[:,'game_id'] = gameid
                AllDFs.append(df)
        
df = pd.concat(AllDFs)     